<a href="https://colab.research.google.com/github/jjennings955/Neural-Network-Notebooks/blob/master/Backpropagation_Object_Oriented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://pastebin.com/Yc7RBFpT

In [0]:
import numpy as np

class Layer(object):
    def __init__(self, parent_layer, *args, **kwargs):
        self.parent = parent_layer
        self.name = kwargs.pop('name', 'unnamed')
        if self.parent:
            self.parent.child = self
        self.child = None
        
      
# Dense/fully connected layer/Affine layer
class Dense(Layer):
    def __init__(self, num_inputs, num_outputs, parent_layer=None, name="Dense", *args, **kwargs):
        super(Dense, self).__init__(parent_layer)
        self.W = np.random.randn(num_outputs, num_inputs)/np.sqrt(num_outputs)
        self.b = np.random.randn(num_outputs, 1)
        
        
    def forward(self, x):
        self.x = x
        return np.matmul(self.W, x) + self.b

    def backward(self, g):
        self.dW = g.dot(self.x.T)
        dX = self.W.T.dot(g)
        self.db = g
        return dX
    def __repr__(self):
        return "Dense(num_inputs={}, num_outputs={}, name={})".format(self.W.shape[0], self.W.shape[0], self.name)
       

class Sigmoid(Layer):
    def __init__(self, parent_layer, *args, **kwargs):
        super(Sigmoid, self).__init__(parent_layer, *args, **kwargs)
        
    def forward(self, x):
        self.out = 1/(1 + np.exp(-x))
        return self.out

    def backward(self, g):
        sigmoid_gradient = self.out*(1 - self.out)
        return sigmoid_gradient*g
    def __repr__(self):
        return "Sigmoid(name={})".format(self.name)
      
class Softmax(Layer):
    def __init__(self, parent_layer, *args, **kwargs):
        super(Softmax, self).__init__(parent_layer, *args, **kwargs)
        
    def forward(self, x):
        e_x = np.exp(x)
        denominator = np.sum(e_x)
        self.output = e_x/denominator
        return self.output

    def backward(self, g):
        jacobian = np.diagflat(self.output) - np.matmul(self.output, self.output.T)
        return np.matmul(jacobian.T, g)
      
    def __repr__(self):
        return "Softmax(name={})".format(self.name)
      
class CrossEntropy(Layer):
    def __init__(self, parent_layer, labels, *args, **kwargs):
        super(CrossEntropy, self).__init__(parent_layer)
        self.labels = labels # We really should have figured out a better way to implement this.
        
    def forward(self, y_hat):
        self.y_hat = y_hat
        self.y = self.labels
        xent = -np.sum(self.y * np.log(self.y_hat + 1e-8))
        return xent
        
    def backward(self, g):
        self.gradient = self.y / self.y_hat
        return self.gradient
      
    def __repr__(self):
        return "CrossEntropy(name={})".format(self.name) 

class Network(Layer):
    def __init__(self, parent_layer):
        super(Network, self).__init__(parent_layer)

    def _root(self):
        obj = self.parent
        while obj.parent != None:
            obj = obj.parent
            self.root = obj
        return self.root

    def forward(self, input):
        obj = self._root()
        current_input = input
        while obj.child:
            out = obj.forward(current_input)
            current_input = out
            print('call', obj, '.forward(X) with output from previous layer,\noutput={}'.format(out))
            print('-----------------------------')
            obj = obj.child

    def backward(self):
        g = 1
        obj = self.parent
    
        while obj:
            g = obj.backward(g)
            print('call', obj, '.backward(g) with g flowing backwards from child layer\ng={}'.format(g))
            print('-----------------------------')
            obj = obj.parent
    
    def __repr__(self):
        output = []
        obj = self._root()
        while obj.child:
          output.append(repr(obj))
          obj = obj.child
        return ' -> '.join(output)
      
      
      
      


# z_1 = Sigmoid(net_1)
# net_2 = Dense(3,3, z_1)
# z_2 = Softmax(net_2)
# loss = CrossEntropy(z_2)



# Testing Dense layer

In [0]:
np.random.seed(1234)
net_1 = Dense(num_inputs=2, num_outputs=3)
print(net_1.W)
print(net_1.b)

# 0.2, -0.3

x = np.float32([[0.2, -0.3]]).T
g = np.float32([[1, 1, 1]]).T
print(net_1.forward(x))
print(net_1.backward(g))

# Testing Sigmoid layer

In [0]:
x = np.float32([[0.0, -5.0, 5.0]]).T
s = Sigmoid(parent_layer=None)
g = np.float32([[1, 1, 1]]).T
print(s.forward(x))
print(s.backward(g))


# Testing Softmax Layer

In [0]:
x = np.float32([[1, 1, 1]]).T
g = np.float32([[-1, 1, 0]]).T
soft = Softmax(None)
print(soft.forward(x))
print(soft.backward(g))

# Testing Cross Entropy layer 

In [0]:
y = np.float32([[1, 0, 0]]).T
x_ent = CrossEntropy(None, labels=y)
y_hat = np.float32([[0.25, 0.4, 0.35]]).T
print(x_ent.forward(y_hat))
print(x_ent.backward(1))


# Creating a network and connecting it all-together using the Network class

In [0]:
y = np.float32([[1, 0, 0]]).T

net_1 = Dense(2, 10, name='dense_1')
z_1 = Sigmoid(net_1, name='sigmoid_1')
net_2 = Dense(10,3, z_1, name='dense_2')
z_2 = Softmax(net_2, name='softmax_outut')
loss = CrossEntropy(z_2, labels=y, name='xent')
network = Network(loss)
print(network)

# Forward Propagation

In [0]:
x = np.float32([[0.2, -0.3]]).T
network.forward(x)

# Compute the backward pass for a single input

In [0]:
network.backward()

# Inspecting the gradient of our loss wrt our weights and biases

In [0]:
net_1.dW

In [0]:
net_1.db

In [0]:
print(net_2.W)
print(net_2.b)